Neural Networks Tensorflow

In [1]:
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [4]:
dataset = pd.read_csv("https://raw.githubusercontent.com/spiderPan/Google-Machine-Learning-Crash-Course/master/data/california_housing_train.csv", sep=",")

In [5]:
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.3,34.2,15.0,5612.0,1283.0,1015.0,472.0,1.5,66900.0
1,-114.5,34.4,19.0,7650.0,1901.0,1129.0,463.0,1.8,80100.0
2,-114.6,33.7,17.0,720.0,174.0,333.0,117.0,1.7,85700.0
3,-114.6,33.6,14.0,1501.0,337.0,515.0,226.0,3.2,73400.0
4,-114.6,33.6,20.0,1454.0,326.0,624.0,262.0,1.9,65500.0


In [8]:
dataset = dataset.reindex(np.random.permutation(dataset.index))

In [9]:
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
4736,-118.1,33.9,18.0,1573.0,396.0,1200.0,365.0,2.9,146900.0
11693,-121.3,38.1,27.0,1004.0,192.0,470.0,192.0,2.9,116700.0
11354,-121.2,38.0,39.0,440.0,83.0,270.0,97.0,6.1,157700.0
10288,-120.0,36.9,11.0,3606.0,699.0,2074.0,644.0,2.7,63300.0
11798,-121.3,37.9,36.0,40.0,10.0,64.0,14.0,2.6,55000.0


In [11]:
dataset['median_house_value'] /= 1000.0
dataset.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


In [24]:
my_feature = dataset[['total_rooms']]

In [26]:
feature_columns = [tf.feature_column.numeric_column("total_rooms")]
feature_columns

[NumericColumn(key='total_rooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [ ]:
targets = dataset['median_house_value']
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)

In [20]:
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [21]:
linear_regressor = tf.estimator.LinearRegressor(feature_columns=feature_columns, optimizer=my_optimizer)

In [22]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    features = {key:np.array(value) for key,value in dict(features).items()} 
    ds = Dataset.from_tensor_slices((features,targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
    features, labels = ds.make_one_shot_iterator().get_next() 
    return features, labels

In [23]:
_ = linear_regressor.train(input_fn = lambda:my_input_fn(my_feature, targets), steps=100)